In [178]:
import nltk
from nltk.stem.porter import PorterStemmer
import json
import numpy as np

import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [179]:
portStemmer = PorterStemmer()
def tokenize(sentence):
    return nltk.word_tokenize(sentence)


def stem(word):    
    return portStemmer.stem(word.lower())


def bag_words(pattern_stentence, all_words):
    sentence = [stem(w) for w in pattern_stentence]
    bag = np.zeros(len(all_words), dtype=np.float32)
    for i, w in enumerate(all_words):
        if w in sentence:
            bag[i] = 1.0
    return bag
    # for word in all_words:
    #     if word in sentence:
    #         bag.append(1.0)
    #     else:
    #         bag.append(0.0)
    # return np.array(bag, dtype=np.float32)

    

In [180]:
sentence = ["WHat", "the", "fuck", "is", "going", "on"]
words = ["fuck", "bye", "what", "on"]
bag_words(sentence, words)

array([1., 0., 1., 1.], dtype=float32)

In [181]:
with open("categories.json", "r") as f:
    intents = json.load(f)

In [182]:
all_words = []
tags = []
pattern_tags = []
for intent in intents["intents"]:
    tag = intent["tag"]
    tags.append(tag)
    pattern_tag = []
    for pattern in intent["patterns"]:
        tokenized_pattern = tokenize(pattern)
        all_words.extend(tokenized_pattern)
        pattern_tags.append((tokenized_pattern, tag))

ignore_chars = ["?", "!", ".", ","]



In [183]:
print(all_words)

['Hi', 'Hey', 'How', 'are', 'you', 'Is', 'anyone', 'there', '?', 'Hello', 'Good', 'day', 'Bye', 'See', 'you', 'later', 'Goodbye', 'Thanks', 'Thank', 'you', 'That', "'s", 'helpful', 'Thank', "'s", 'a', 'lot', '!', 'Which', 'items', 'do', 'you', 'have', '?', 'What', 'kinds', 'of', 'items', 'are', 'there', '?', 'What', 'do', 'you', 'sell', '?', 'Do', 'you', 'take', 'credit', 'cards', '?', 'Do', 'you', 'accept', 'Mastercard', '?', 'Can', 'I', 'pay', 'with', 'Paypal', '?', 'Are', 'you', 'cash', 'only', '?', 'How', 'long', 'does', 'delivery', 'take', '?', 'How', 'long', 'does', 'shipping', 'take', '?', 'When', 'do', 'I', 'get', 'my', 'delivery', '?', 'Tell', 'me', 'a', 'joke', '!', 'Tell', 'me', 'something', 'funny', '!', 'Do', 'you', 'know', 'a', 'joke', '?']


In [184]:
# applying stemming and remove duplicate words by type-casting the list into a set
all_words = set([stem(w) for w in all_words if w not in ignore_chars])

In [185]:
print(all_words)

{'can', 'do', 'good', 'hi', 'what', 'tell', 'someth', 'know', 'hey', 'day', 'with', 'of', 'later', 'have', 'i', 'goodby', 'which', 'joke', 'get', 'deliveri', 'mastercard', 'cash', 'onli', 'when', 'help', 'card', 'item', "'s", 'thank', 'accept', 'sell', 'pay', 'bye', 'credit', 'is', 'long', 'a', 'how', 'that', 'lot', 'doe', 'there', 'anyon', 'see', 'take', 'kind', 'my', 'funni', 'ship', 'are', 'you', 'me', 'hello', 'paypal'}


In [186]:
all_words = sorted(all_words)
print(all_words)

["'s", 'a', 'accept', 'anyon', 'are', 'bye', 'can', 'card', 'cash', 'credit', 'day', 'deliveri', 'do', 'doe', 'funni', 'get', 'good', 'goodby', 'have', 'hello', 'help', 'hey', 'hi', 'how', 'i', 'is', 'item', 'joke', 'kind', 'know', 'later', 'long', 'lot', 'mastercard', 'me', 'my', 'of', 'onli', 'pay', 'paypal', 'see', 'sell', 'ship', 'someth', 'take', 'tell', 'thank', 'that', 'there', 'what', 'when', 'which', 'with', 'you']


In [187]:
tags = sorted(set(tags))
print(tags)

['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [188]:
X_train = []
Y_train = []
for x in pattern_tags:
    bag = bag_words(x[0], all_words)   # sentence, all_words
    X_train.append(bag)
    label = tags.index(x[1])
    Y_train.append(label)

X_train = np.array(X_train)
Y_train = np.array(Y_train, dtype=np.int64)
print(type(X_train))
# X_train = X_train.type(torch.LongTensor)

<class 'numpy.ndarray'>


In [189]:
class ChatDataSet(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = torch.from_numpy(X_train)
        self.y_data = torch.from_numpy(Y_train)
        print(type(self.x_data))
        print(type(self.x_data[0]))
        print(self.x_data)
        print(type(self.y_data))
        print(self.y_data)

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

batch_size = 8

dataset = ChatDataSet()
train_loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)


<class 'torch.Tensor'>
<class 'torch.Tensor'>
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 1.],
        ...,
        [0., 1., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 1., 0.,  ..., 0., 0., 1.]])
<class 'torch.Tensor'>
tensor([3, 3, 3, 3, 3, 3, 2, 2, 2, 6, 6, 6, 6, 4, 4, 4, 5, 5, 5, 5, 0, 0, 0, 1,
        1, 1])


In [190]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, classes):
        super(Model, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.l3 = nn.Linear(hidden_size, classes)

        self.relu = nn.ReLU()

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)

        return out

In [191]:
print(len(tags))

7


In [192]:
INPUT_SIZE = len(X_train[0])
OUTPUT_SIZE = len(tags)
HIDDEN_SIZE = 8
BATCH_SIZE = 8

LEARN_RATE = 0.001
EPOCHS = 1000

print(INPUT_SIZE, len(all_words))
print(OUTPUT_SIZE, tags)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Model(INPUT_SIZE, HIDDEN_SIZE, OUTPUT_SIZE).to(device)

54 54
7 ['delivery', 'funny', 'goodbye', 'greeting', 'items', 'payments', 'thanks']


In [193]:
# Optimization 

xEntropyLoss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARN_RATE)

for epoch in range(EPOCHS):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(device)

        # forward pass
        
        outputs = model(words) 
            
        loss = xEntropyLoss(outputs, labels)

        #Back propogation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
        print(f"epoch: {epoch+1} / {EPOCHS}, loss: {loss.item()} ")


print(f"final loss: {loss.item()}")

epoch: 100 / 1000, loss: 1.0150247812271118 
epoch: 200 / 1000, loss: 0.1493098884820938 
epoch: 300 / 1000, loss: 0.00857056025415659 
epoch: 400 / 1000, loss: 0.0026586707681417465 
epoch: 500 / 1000, loss: 0.0016610106686130166 
epoch: 600 / 1000, loss: 0.0007009432883933187 
epoch: 700 / 1000, loss: 0.00014756870223209262 
epoch: 800 / 1000, loss: 0.0020305849611759186 
epoch: 900 / 1000, loss: 0.0004318160645198077 
epoch: 1000 / 1000, loss: 0.0007990890881046653 
final loss: 0.0007990890881046653
